<h1>Scientific coding bootcamp notebook 2: Plotting and linear algebra.</h1>
Julia is designed from head to toe for scientific computing.  Much of its power comes from the state of the art packages that are part of the Julia ecosystem.  We'll acquaint ourselves with a few of those packages in this notebook. 

<h2>Plotting</h2>

In [ ]:
using Plots, LinearAlgebra

In [ ]:
# Basic plotting
x = 0:0.1:10      # This is called a range.  
y = x .^ 2
plot(x,y,label="y=x^2")

In [ ]:
# There are two ways to plot multiple lines.  The first is to simply pass a vector of things to plot:
plot(x,[y,x.^3]; labels = ["y=x^2" "y=x^3"], title="This is a title")

In [ ]:
# The second is to use the function "plot!", which updates a plot:
p = plot(x,y,label="y=x^2")
plot!(p,x,x.^4,label="y=x^4",ylim=(0,100))

</hline>
<h1>Linear algebra</h1>
<h3>SVD and pinv</h3>

We now come to linear algebra.  The power of linear algebra is that it lets you reason geometrically about algebra, and it lets you reason algebraically about geometry.  We begin with the very useful singular value decomposition (SVD) and the related pseudo-inverse.

SVD breaks an arbitrary matrix into a composition of a rotation, a stretching (diagonal matrix), and another rotation.  More explicitly, SVD of an arbitrary matrix M is given by U*S*V', where U and V are rotation matrices, S is diagonal, and ' denotes the conjugate-transpose.  The singular values tell you a lot about the matrix, such as whether it is invertible, and they are usually sorted from largest to smallest.  (Remarkably, this works even if M is not a square matrix.  Question: What do you think it means for a non-square matrix to be "diagonal"?)

The pseudoinverse is the "closest to an inverse you can get".  If a matrix M is invertable, the pseudoinverse pinv(M) is just the regular inverse M^{-1}.  If M is not invertable but it is diagonal, then the pinv(M) is computed by inverting each non-zero diagonal element and leaving every other element zero.  In the general case, if the SVD of M is U*S*V', the pseudoinverse of M is V*pinv(S)*transpose(U). 

In [ ]:
# Try to interpret geometrically what this matrix does.
M = [0 -2 0 ; 2 0 0 ; 0 0 0]
F = svd(M)

In [ ]:
pinv(M)

Below are some problems to get you familiar with pinv and SVD. 

In [ ]:
# Compute the pseudoinverse of the pseudoinverse of M, and interpret it. 

In [ ]:
# Extract the diagonal factor of svd(M), and turn it into a diagonal matrix. 

In [ ]:
# Find a matrix for which both rotations in the SVD are the identity matrix. 

In [ ]:
# Find a non-diagonal matrix for which the SVD diagonal factor has exactly one non-zero entry. 

In [ ]:
# Compute the determinant of a matrix using the diagonal factor of its SVD. 

In [ ]:
# Compute the SVD of a non-square matrix.  What are the shapes of the factors?  Why does this make sense?

<h3>Least squares</h3>

A very useful application of pinv is for computing linear least squares fits.  Suppose we have data with x-coordinates xs and y coordinates ys.  A least squares fit is a slope A and intercept b such that (ys - A*xs - b) is pretty small.  This can be rearranged into 

ys ≈ [xs ones(length(xs))] * [A, b], 

or 

[A,b] = pinv([xs ones(length(xs))]) * ys.

In [ ]:
xs = sort(rand(100))
ys = sort(rand(100))
A,b = pinv([xs ones(length(xs))]) * ys
println(A)
println(b)
scatter(xs,ys,label="Data")
plot!(xs,A*xs .+ b,label="Fit")

There's a convenient shorthand for the pseudoinverse, using the operator \. 

In [ ]:
A,b = [xs ones(length(xs))] \ ys

<h3>Function spaces</h3>

Arguably the most powerful application of linear algebra is to vector spaces of functions.  To see how this works, we will start with spaces of polynomials.  Consider the set $P_N$ of all polynomials up to some fixed degree $N$.  We will take $N=4$ for concreteness in examples to follow.  An example of an element of $P_4$ is $p(x) = 12 - 4x/3 + \pi^2 x^4$. 

**Question:** What is a basis for $P_4$? What is the dimension of $P_4$?

A downside of the "obvious" basis for $P_4$ is that it's not easy to figure out how to express a given function in terms of the basis polynomials.  For instance, if I give you a list of numbers like 
```
fs=[0.0625, 0.0256, 0.0081, 0.0016, 0.0001, 0.0, 0.0001, 0.0016, 0.0081, 0.0256, 0.0625]
```
and tell you that these are the values of some polynomial sampled at the points `0.0:0.1:1.0`, it's probably not immediately clear how to recover the polynomial.  The key to doing so is to define an inner product on this vector space. 

An inner product is an abstract version of a dot product.  A typical example of an inner product of two real functions $f(x)$ and $g(x)$ is 
$$
\langle f | g \rangle = \int_0^1 f(x) g(x) dx.
$$
Given a basis $B$ and an arbitrary function $f\in P_N$, we can always write $f(x)=\sum_{p\in B} c_p p(x)$ for some coefficients $c_p$.  We can use an inner product and some linear algebra to figure out what the coefficients are.  
$$
\langle q | f\rangle = \sum_{p\in B} \langle q | p\rangle c_p = \Bigl[ \langle q | p_0 \rangle, q | p_1 \rangle, \langle q | p_2 \rangle, \langle q | p_3 \rangle, \langle q | p_4 \rangle \Bigr] \begin{bmatrix}
           c_0 \\ c_1 \\ c_2 \\ c_3 \\ c_4
         \end{bmatrix}
$$
Stacking together the equation for each $q \in B$ gives a matrix equation
$$
\begin{bmatrix}
           \langle p_0 | f\rangle \\ \langle p_1 | f\rangle \\ \langle p_2 | f\rangle \\ \langle p_3 | f\rangle \\ \langle p_4 | f\rangle 
\end{bmatrix} = 
\begin{bmatrix}
\langle p_0 | p_0 \rangle & \langle p_0 | p_1 \rangle & \langle p_0 | p_2 \rangle & \langle p_0 | p_3 \rangle & \langle p_0 | p_4 \rangle \\
\langle p_1 | p_0 \rangle & \langle p_1 | p_1 \rangle & \langle p_1 | p_2 \rangle & \langle p_1 | p_3 \rangle & \langle p_1 | p_4 \rangle \\
\langle p_2 | p_0 \rangle & \langle p_2 | p_1 \rangle & \langle p_2 | p_2 \rangle & \langle p_2 | p_3 \rangle & \langle p_2 | p_4 \rangle \\
\langle p_3 | p_0 \rangle & \langle p_3 | p_1 \rangle & \langle p_3 | p_2 \rangle & \langle p_3 | p_3 \rangle & \langle p_3 | p_4 \rangle \\
\langle p_4 | p_0 \rangle & \langle p_4 | p_1 \rangle & \langle p_4 | p_2 \rangle & \langle p_4 | p_3 \rangle & \langle p_4 | p_4 \rangle
\end{bmatrix}
\begin{bmatrix}
           c_0 \\ c_1 \\ c_2 \\ c_3 \\ c_4 
\end{bmatrix}
$$
By matrix inversion we can get the coefficients in terms of the inner products $\langle q | f\rangle$. 

In [ ]:
# Complete the following code to determine the polynomial coefficients associated to the data points fs. 
fs = [0.0625, 0.0256, 0.0081, 0.0016, 0.0001, 0.0, 0.0001, 0.0016, 0.0081, 0.0256, 0.0625]
xs = 0.0:0.1:1.0

function innerProduct01(f::Vector{Float64},g::Vector{Float64})
    if length(f) != length(g) 
        error("Inputs have different lengths.")
    end
    N = length(f) 
    dx = 1.0/(N-1)
    # Your code here.  Compute the integral of fg from 0 to 1.  You may use any integration rule you like.  
    # You can find examples of integration rules in Numerical Recipes 4.1.3. 
end

innerProds = [innerProduct01(fs,(x-> x^k).(xs)) for k=0:4]
coeffs = inv([innerProduct01((x-> x^j).(xs),(x-> x^k).(xs)) for j=0:4, k=0:4]) * innerProds  # Your code here. 

<h4>Polynomial fitting.</h4>

You can use the same method as above to fit a fourth order polynomial to data which is *not* polynomial.  E.g. noisy data. 

In [ ]:
noisyfs = [0.0625, 0.0256, 0.0081, 0.0016, 0.0001, 0.0, 0.0001, 0.0016, 0.0081, 0.0256, 0.0625] + (rand(11) .- 0.5) * 0.02
noisyInnerProds = [innerProduct01(noisyfs,(x-> x^k).(xs)) for k=0:4]
noisyCoeffs = inv([innerProduct01((x-> x^j).(xs),(x-> x^k).(xs)) for j=0:4, k=0:4]) * noisyInnerProds  # Your code here. 
println(noisyCoeffs)
# Your code here.  Make a scatter plot of xs vs. noisyfs. 
plot!((x-> sum(noisyCoeffs[j+1]*x^j for j=0:4)), xlim=(0,1))

In practice, when you do curve fitting of this sort you should usually use a dedicated fitting package. 

<h3>Gaussian beam propagation</h3>

We will apply the above ideas to a space of two-dimensional functions which model laser beam propagation. We begin by defining the Hermite-Gaussian modes, which are the most basic shapes that laser beams can assume.  

In [ ]:
using SpecialPolynomials: Hermite, basis
# Define a 64x64 computational grid
N = 64
const grid = (-sqrt(N)/2:sqrt(1/N):sqrt(N)/2-sqrt(1/N), -sqrt(N)/2:sqrt(1/N):sqrt(N)/2-sqrt(1/N))

function HGMode(grid, n::Int, m::Int; waist::Float64=1.0, z::Float64=0.0, lam::Float64=1.0)
    return HG1D(grid[1],n;waist=waist,z=z,lam=lam) .* transpose(HG1D(grid[2],m;waist=waist,z=z,lam=lam))
end

function HG1D(grid, n::Int; waist::Float64=1.0, z::Float64=0.0, lam::Float64=1.0)
    zr = pi * waist^2 / lam
    q = z + im*zr
    w = waist*sqrt(1+(z/zr)^2)
    prefactor = sqrt(sqrt(2/pi)/(2^n * factorial(n) * waist)) * sqrt(1/(1-im*z/zr)) * sqrt(-conj(q)/q)^n
    polyfactor = basis(Hermite,n).(grid * sqrt(2) ./ w)
    expfactor = exp.(-2*pi*im * grid.^2 / (2*lam*q))
    return prefactor .* polyfactor .* expfactor
end

In [ ]:
# Why do we need to take abs here? 
heatmap(abs.(HGMode(grid,10,0)).^2)

Hermite-Gaussian modes $H_{n,m}(x,y)$ form a basis for 2D functions.  This means that any 2D function can be written as a sum of the form
$$
\sum_{n,m} c_{n,m} H_{n,m}(x,y).
$$
How do we determine the coefficients $c_{n,m}$?  We could do the same matrix inversion as we did above, but HG modes have the special property of *orthonormality* which greatly simplifies the analysis.  The integral of a product of two HG modes (one of them complex conjugated) is zero if the modes are different and 1 if they are the same,
$$
\int_{\mathbb{R}^2} H_{n,m}(x,y) H^{*}_{p,q}(x,y) \; dx \; dy = \delta_{np} \delta_{mq}.
$$

If some function $f(x,y)$ equals $\sum_{n,m} c_{n,m} H_{n,m}(x,y)$ for some choice of coefficients $c_{n,m}$, by multiplying both expressions against $H^{*}_{p,q}$ and integrating, we find
$$
c_{p,q} = \int_{\mathbb{R}^2} f(x,y) H^{*}_{p,q}(x,y) \; dx \; dy.
$$
You will implement this *mode decomposition* below for a function (represented as a matrix) sampled on the same grid as above. 

In [ ]:
# Finish the code below to implement the overlap integral in the mode decomposition coefficient formula. 
# Numerical Recipes 4.1.3 has relevant integration formulas, though you can probably devise your own.
function HGOverlap(grid,f::Matrix{T},n::Int,m::Int; waist::Float64=1.0, z::Float64=0.0, lam::Float64=1.0) where {T<:Number}
    mode = HGMode(grid, n, m; waist=waist, z=z, lam=lam)
    integrand = f .* conj.(mode)
    # Your code here. 
end

LG01 = [exp(-x^2-y^2) * (1-2x^2-2y^2) for x in grid[1], y in grid[2]] * sqrt(2/pi)
coeffs = [HGOverlap(grid,LG01,n,m) for n=0:5,m=0:5]

In [ ]:
# If you coded the above correctly, the sum of squared absolute values of the coefficients should be one.  
sum(abs.(coeffs).^2) |> println
heatmap(abs.(coeffs).^2)        # This visually shows where the coefficients are large. 

In [ ]:
# Examine the form of the profile LG01. Then use the coefficients you found above and the HGMode function evaluated at z=4 
# to determine what a laser with initial profile LG01 will look like after propagating distance 4. 
heatmap(abs.(LG01).^2)

In [ ]:
heatmap( #= Your code here.  Determine the form of the beam at z=4. =# )